## Does a dataset trained on Amharic perform better than regular XLMR?


We're just looking for performance better than 68% so maybe adding more languages in encourages some transfer learning. Let's add the other languages in and see if there's performance improvements.

In [41]:
# Import tfds and tensorflow
import tensorflow as tf
import tensorflow_datasets as tfds
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd

In [42]:
import torch
from accelerate import Accelerator
accelerator = Accelerator()

### 1.1. Load just Amharic Data

We're going to load everything upfront so that we have it all more or less ready. If it's a generation run, then we're looking to use a non-augmented dataset and see what the model can and can't recognize and then use that to generate new comments. If it isn't a generation run, then we're loading those augmented comments and testing against the held back 500

In [43]:
GENERATION_RUN = False

################################
# for generation
################################

if(GENERATION_RUN):
    amharic_data = pd.read_csv('data/Amharic/amharic_train_29.5k.csv')
    amharic_train, amharic_test = train_test_split(amharic_data)
else: # for TESTING generated comments
    amharic_train = pd.read_csv('data/Amharic/new_amharic_mono_3.csv')
    amharic_test = pd.read_csv('data/Amharic/amharic_test_reserve_500.csv')

In [44]:
amharic_train.head()

,id,tweet,subtask_a
0,3321.0,ድልብ ማይም,NOT
1,22116.0,ካልተቀበሉ ጡረታ ይዉጡ ፖለቲካ በቃቸዉ,OFF
2,12776.0,በቀለ ገርባ ነቀለ ገለባ አቤት ውሸት,OFF
3,22938.0,እሄ ነው ለአገር እና ለህዝብ ማሰብ,NOT
4,19580.0,ስለፍትህ ስርአቱ አይናችሁን በጨው ተታጥባችሁ አስተያየት ስትሰጡ ሳቄ ነው...,OFF


In [45]:
MODEL_TYPE = 'Davlan/xlm-roberta-base-finetuned-amharic'

L_RATE = 0.000005 #1e-5 # need to change this potentially
MAX_LEN = 128

NUM_EPOCHS = 3
BATCH_SIZE = 16

In [46]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

# xlm-roberta-large
print('Loading XLMRoberta tokenizer...')
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_TYPE)
print('Tokenizer loaded')

Loading XLMRoberta tokenizer...
Tokenizer loaded


In [47]:
import torch

def create_tensors( all_training_sets ):
    
    _all_labels = []
    _all_input_ids = []
    _all_attn_masks = []
    
    for t_set in all_training_sets:
        new_ids, new_masks, new_labels = tokenize_dataset(t_set)
        _all_input_ids = [*_all_input_ids, *new_ids]
        _all_attn_masks = [*_all_attn_masks, *new_masks]
        _all_labels = [*_all_labels, *new_labels]
        
    # Convert each Python list of Tensors into a 2D Tensor matrix.
    _all_input_ids = torch.cat(_all_input_ids, dim=0)
    _all_attn_masks = torch.cat(_all_attn_masks, dim=0)

    # Cast the labels list to a Tensor.
    _all_labels_tensor = torch.tensor(_all_labels)
    
    return _all_input_ids, _all_attn_masks, _all_labels_tensor, _all_labels

def tokenize_dataset( training_set ):
    
    _labels = []
    _ids = []
    _masks = []
    
    # look through all records
    for index, row in training_set.iterrows():
        encoded_dict = tokenizer.encode_plus(
                    row['tweet'], # Sentence to encode.
                    add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
                    max_length = MAX_LEN,           # Pad or truncate.
                    pad_to_max_length = True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',          # Return pytorch tensors.
                   )

        # Add this example to our lists.
        _ids.append(encoded_dict['input_ids'])
        _masks.append(encoded_dict['attention_mask'])
        if(row['subtask_a'] == 'NOT'):
            _labels.append(0)
        else:
            _labels.append(1)
    
    return _ids, _masks, _labels

datasets = [amharic_train]
all_input_ids, all_attn_masks, all_labels_tensor, all_labels = create_tensors(datasets)


print('   DONE. {:,} examples.'.format(len(all_labels)))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/joshnoble/.local/share/virtualenvs/joshnoble-TX97Gyyd/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2105: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


   DONE. 65,671 examples.


In [48]:
datasets = [amharic_test]
test_input_ids_amharic, test_attn_masks_amharic, test_labels_amharic_tensor, test_labels_amharic = create_tensors(datasets)

print('   DONE. {:,} examples.'.format(len(test_labels_amharic)))

   DONE. 500 examples.


In [49]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Specify our batch size.
batch_size = 16

# Construct a TensorDataset from the encoded examples.
prediction_dataset = TensorDataset(test_input_ids_amharic, test_attn_masks_amharic, test_labels_amharic_tensor)

# And a dataloader for handling batching.
prediction_dataloader = DataLoader(prediction_dataset, batch_size=batch_size)

In [50]:
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(all_input_ids, all_attn_masks, all_labels_tensor)

# Create a 90-10 train-validation split. Calculate the number of samples to 
# include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create the DataLoaders for our training sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )




## Some Helper Functions

In [51]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def good_update_interval(total_iters, num_desired_updates):
    exact_interval = total_iters / num_desired_updates
    order_of_mag = len(str(total_iters)) - 1
    round_mag = order_of_mag - 1
    update_interval = int(round(exact_interval, -round_mag))

    # Don't allow the interval to be zero!
    if update_interval == 0:
        update_interval = 1

    return update_interval

## Get the model

In [52]:
from transformers import AdamW

#can use accelerate to spread across multiple GPUs
USING_ACCELERATE = False

# we're going 2 labels: offensive and not using our new model that we've finetuned (again)
xlmr_model = XLMRobertaForSequenceClassification.from_pretrained("models/retune_xlmr", num_labels=2)

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(xlmr_model.parameters(),
                  lr = 5e-5, #5e-6, # args.learning_rate
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

from transformers import get_linear_schedule_with_warmup

# Number of training epochs.
epochs = 5

# Total number of training steps is [number of batches] x [number of epochs]. 
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,  num_warmup_steps = 0, num_training_steps = total_steps)

if(USING_ACCELERATE):

    xlmr_model, optimizer = accelerator.prepare(xlmr_model, optimizer)
    device = accelerator.device
    
else:

    print('Loading model to GPU...')

    # Connect to the GPU.
    device = torch.device('cuda:3')

    # Report what GPU we were granted.
    print('  GPU:', torch.cuda.get_device_name(0))

    # Copy the initial model weights to the GPU.
    desc = xlmr_model.to(device)

    print('    DONE.')

Some weights of the model checkpoint at models/retune_xlmr were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at models/retune_xlmr and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.wei

Loading model to GPU...
  GPU: Tesla K80
    DONE.


In [53]:
import random
import numpy as np

# Code heavily based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    xlmr_model.train()

    # Pick an interval on which to print progress updates.
    update_interval = good_update_interval(
                total_iters = len(train_dataloader), 
                num_desired_updates = 10
            )

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update.
        if (step % update_interval) == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        xlmr_model.zero_grad()        

        loss, logits = xlmr_model.forward(b_input_ids, 
                             attention_mask=b_input_mask, 
                             labels=b_labels, return_dict=False)

        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        if(USING_ACCELERATE):
            accelerator.backward(loss)
        else:
            loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(xlmr_model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
 
    # ========================================
    #               Validation
    # ========================================
    print("")
    print("Running Validation...")

    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    xlmr_model.eval()
    # Tracking variables 
    total_eval_loss = 0
    predictions, true_labels = [], []

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        
            (loss, logits) = xlmr_model.forward(b_input_ids, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels, return_dict=False)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Calculate the accuracy for this batch of test sentences.

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Store predictions and true labels
        predictions.append(logits)
        true_labels.append(label_ids)
    
    # Combine the results across all batches. 
    flat_predictions = np.concatenate(predictions, axis=0)
    flat_true_labels = np.concatenate(true_labels, axis=0)

    # For each sample, pick the label (0, 1, or 2) with the highest score.
    predicted_labels = np.argmax(flat_predictions, axis=1).flatten()

    # Calculate the validation accuracy.
    val_accuracy = (predicted_labels == flat_true_labels).mean()

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...
  Batch   400  of  3,694.    Elapsed: 0:05:22.
  Batch   800  of  3,694.    Elapsed: 0:10:42.
  Batch 1,200  of  3,694.    Elapsed: 0:16:02.
  Batch 1,600  of  3,694.    Elapsed: 0:21:21.
  Batch 2,000  of  3,694.    Elapsed: 0:26:39.
  Batch 2,400  of  3,694.    Elapsed: 0:31:58.
  Batch 2,800  of  3,694.    Elapsed: 0:37:17.
  Batch 3,200  of  3,694.    Elapsed: 0:42:35.
  Batch 3,600  of  3,694.    Elapsed: 0:47:54.

  Average training loss: 0.69
  Training epcoh took: 0:49:09

Running Validation...
  Accuracy: 0.51
  Validation Loss: 0.69
  Validation took: 0:01:35

======== Epoch 2 / 5 ========
Training...
  Batch   400  of  3,694.    Elapsed: 0:05:19.
  Batch   800  of  3,694.    Elapsed: 0:10:37.
  Batch 1,200  of  3,694.    Elapsed: 0:15:55.
  Batch 1,600  of  3,694.    Elapsed: 0:21:13.
  Batch 2,000  of  3,694.    Elapsed: 0:26:32.
  Batch 2,400  of  3,694.    Elapsed: 0:31:50.
  Batch 2,800  of  3,694.    Elapsed: 0:37:08.
  Batch 3

## 4.2. Run Prediction

Run all of our Amharic test samples through our model to get predictions!

In [56]:
# Prediction on test set
print('Predicting labels for {:,} test examples...'.format(len(test_labels_amharic)))

# Put model in evaluation mode
xlmr_model.eval()

# Tracking variables 
predictions , true_labels = [], []

count = 0

# Predict 
for batch in prediction_dataloader:
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = xlmr_model(b_input_ids, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 500 test examples...
    DONE.


In [57]:
# Combine the results across all batches. 
flat_predictions = np.concatenate(predictions, axis=0)
flat_true_labels = np.concatenate(true_labels, axis=0)

# For each sample, pick the label (0, 1) with the highest score.
predicted_labels = np.argmax(flat_predictions, axis=1).flatten()

In [58]:
# Tally the number of correct predictions to get our accuracy.
accuracy = (predicted_labels == flat_true_labels).mean()

print('XLM-R Prediction accuracy for amharic: {:.3f}'.format(accuracy))

XLM-R Prediction accuracy for amharic: 0.496


With a learning rate of 5e-6, non-generated comments: 0.666

With a learning rate of 5e-6, non-matched generated comments: 0.672

In [59]:
amt = pd.Series(amharic_test["tweet"], dtype="string")
am_df = pd.DataFrame(flat_predictions, columns = ["PRED_NON", "PRED_OFF"])
am_df["PRED"] = predicted_labels
am_df["LABEL"] = flat_true_labels
am_df["TEXT"] = amt.values
am_df.head()

if(GENERATION_RUN):
    am_df.to_csv("am_mono_pred_295_CLEAN.csv")

In [60]:
from sklearn.metrics import f1_score

f1_score(am_df["LABEL"], am_df["PRED"])

0.6631016042780749

With synthetic generated comments: 0.688

With synthetic generated comments and a fine-tuned dataset: 0.6626262626262627